**TF-IDF & Random Forest**

In [ ]:
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
ile_path = "/content/Cleaned Smart Email Dataset.csv"
df = pd.read_csv(file_path)

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)  #
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [ ]:
df['Cleaned_Content'] = df['Email Content'].apply(clean_text)

In [ ]:
X = df['Cleaned_Content']
y = df['Category']

In [ ]:
vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf = vectorizer.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.97
Classification Report:
                             precision    recall  f1-score   support

                         -       0.97      1.00      0.98       525
                  Business       0.00      0.00      0.00         3
                   Finance       0.00      0.00      0.00         2
        Order Confirmation       0.00      0.00      0.00         1
Order Confirmation/Updates       0.00      0.00      0.00         6
                  Personal       1.00      0.33      0.50         3
               Promotional       1.00      0.50      0.67         2
                   Updates       0.00      0.00      0.00         1

                  accuracy                           0.97       543
                 macro avg       0.37      0.23      0.27       543
              weighted avg       0.95      0.97      0.96       543



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE, RandomOverSampler


file_path = "/content/Cleaned Smart Email Dataset.csv"
df = pd.read_csv(file_path)


df = df[df['Category'] != '-']

def clean_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'\d+', '', text)
        text = text.translate(str.maketrans('', '', string.punctuation))
        text = re.sub(r'\s+', ' ', text).strip()
    return text


df['Cleaned_Content'] = df['Email Content'].apply(clean_text)


X = df['Cleaned_Content']
y = df['Category']


vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X_tfidf = vectorizer.fit_transform(X)


if y.value_counts().min() < 2:
    X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)
    print("Warning: A class has only one sample, stratification is disabled.")
else:
    X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42, stratify=y)


class_counts = y_train.value_counts()
min_class_count = class_counts.min()


if min_class_count < 2:
    print("Using RandomOverSampler due to extremely small class sizes.")
    resampler = RandomOverSampler(random_state=42)
else:
    print("Using SMOTE for balancing.")
    resampler = SMOTE(random_state=42, k_neighbors=min(2, min_class_count - 1))


X_train_resampled, y_train_resampled = resampler.fit_resample(X_train, y_train)


model = RandomForestClassifier(n_estimators=200, random_state=42, class_weight="balanced")
model.fit(X_train_resampled, y_train_resampled)


y_pred = model.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", classification_report(y_test, y_pred))

Using RandomOverSampler due to extremely small class sizes.
Accuracy: 0.59
Classification Report:
                             precision    recall  f1-score   support

                  Business       0.40      0.80      0.53         5
                   Finance       0.00      0.00      0.00         2
Order Confirmation/Updates       0.00      0.00      0.00         1
                  Personal       0.67      0.67      0.67         3
               Promotional       1.00      0.75      0.86         4
     Subscription Renewals       0.00      0.00      0.00         1
    Transaction & Security       1.00      1.00      1.00         1

                  accuracy                           0.59        17
                 macro avg       0.44      0.46      0.44        17
              weighted avg       0.53      0.59      0.54        17



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import pandas as pd


file_path = "/content/Cleaned Smart Email Dataset.csv"
df = pd.read_csv(file_path)

df['Cleaned_Content'] = df['Email Content'].str.replace('-', '', regex=False)

df.to_csv("/content/Cleaned_Email_Dataset.csv", index=False)

print(df[['Email Content', 'Cleaned_Content']].head())

                                       Email Content  \
0  [image: Google]\nArchive of Google data reques...   
1  ----------------------------------------------...   
2  CarGurus Check out these listings near you ---...   
3  All your faves for $2, $3, or $5, all day long...   
4  Discover the menu\n\nView this email online.\n...   

                                     Cleaned_Content  
0  [image: Google]\nArchive of Google data reques...  
1  \nTemu\n\n\nTo properly view the full message ...  
2  CarGurus Check out these listings near you  Ca...  
3  All your faves for $2, $3, or $5, all day long...  
4  Discover the menu\n\nView this email online.\n...  


In [ ]:
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE


file_path = "/content/Cleaned Smart Email Dataset.csv"
df = pd.read_csv(file_path)

def clean_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'\d+', '', text)
        text = text.translate(str.maketrans('', '', string.punctuation))
        text = re.sub(r'\s+', ' ', text).strip()
        text = text.replace("-", "")
    return text

df['Cleaned_Content'] = df['Email Content'].apply(clean_text)


def categorize_email(content):
    if any(word in content for word in ['discount', 'offer', 'sale', 'deal', 'promo', 'limited', 'coupon']):
        return 'Promotional'
    elif any(word in content for word in ['meeting', 'project', 'client', 'work', 'business', 'conference', 'office']):
        return 'Business'
    elif any(word in content for word in ['transaction', 'security', 'password', 'verification', 'login', 'fraud', 'alert']):
        return 'Transaction & Security'
    elif any(word in content for word in ['family', 'friend', 'personal', 'vacation', 'birthday', 'social']):
        return 'Personal'
    elif any(word in content for word in ['finance', 'bank', 'loan', 'credit', 'investment', 'money', 'billing']):
        return 'Finance'
    elif any(word in content for word in ['update', 'change', 'news', 'announcement', 'latest', 'info']):
        return 'Updates'
    elif any(word in content for word in ['subscription', 'renewal', 'membership', 'subscribe', 'unsubscribed']):
        return 'Subscription'
    else:
        return 'Updates'


df['Category'] = df['Cleaned_Content'].apply(categorize_email)

X = df['Cleaned_Content']
y = df['Category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


min_class_count = y_train.value_counts().min()
if min_class_count > 1:
    smote = SMOTE(random_state=42, k_neighbors=min(5, min_class_count - 1))
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train)
else:
    print("Class imbalance too severe. Skipping SMOTE.")
    X_train_resampled, y_train_resampled = X_train_tfidf, y_train


model = RandomForestClassifier(n_estimators=200, random_state=42, class_weight="balanced")
model.fit(X_train_resampled, y_train_resampled)


y_pred = model.predict(X_test_tfidf)


accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.88
Classification Report:
                         precision    recall  f1-score   support

              Business       0.85      0.81      0.83        90
               Finance       0.67      0.67      0.67         9
              Personal       0.81      0.74      0.77        23
           Promotional       0.92      0.93      0.92       238
          Subscription       0.65      0.80      0.72        30
Transaction & Security       0.94      0.90      0.92        51
               Updates       0.93      0.91      0.92       102

              accuracy                           0.88       543
             macro avg       0.82      0.82      0.82       543
          weighted avg       0.89      0.88      0.88       543



 **LSTM**

In [ ]:
import pandas as pd
import numpy as np
import re
import string
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

file_path = "/content/Cleaned Smart Email Dataset.csv"
df = pd.read_csv(file_path)


def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\s+', ' ', text).strip()
    return text


df['Cleaned_Content'] = df['Email Content'].fillna("").apply(clean_text)


label_encoder = LabelEncoder()
df['Category_Label'] = label_encoder.fit_transform(df['Category'])


X = df['Cleaned_Content'].tolist()
y = df['Category_Label'].values


max_words = 5000
max_len = 100

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_sequences, maxlen=max_len, padding='post', truncating='post')


X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)


# Define the LSTM model
# Removed the duplicated model definition and corrected indentation
model = Sequential([
    Embedding(input_dim=max_words, output_dim=128, input_length=max_len),
    SpatialDropout1D(0.2),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(64, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


epochs = 5
batch_size = 32

history = model.fit(
    X_train, y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test),
    verbose=1
)


loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")


model_path = "/mnt/data/lstm_email_classifier.h5"
model.save(model_path)
print(f"LSTM model saved to {model_path}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
68/68 ━━━━━━━━━━━━━━━━━━━━ 22s 244ms/step - accuracy: 0.8820 - loss: 1.0482 - val_accuracy: 0.9669 - val_loss: 0.2236
Epoch 2/5
68/68 ━━━━━━━━━━━━━━━━━━━━ 15s 169ms/step - accuracy: 0.9727 - loss: 0.1780 - val_accuracy: 0.9669 - val_loss: 0.2296
Epoch 3/5
68/68 ━━━━━━━━━━━━━━━━━━━━ 21s 178ms/step - accuracy: 0.9705 - loss: 0.1803 - val_accuracy: 0.9669 - val_loss: 0.2168
Epoch 4/5
68/68 ━━━━━━━━━━━━━━━━━━━━ 19s 163ms/step - accuracy: 0.9641 - loss: 0.2104 - val_accuracy: 0.9669 - val_loss: 0.2186
Epoch 5/5
68/68 ━━━━━━━━━━━━━━━━━━━━ 12s 172ms/step - accuracy: 0.9698 - loss: 0.1799 - val_accuracy: 0.9669 - val_loss: 0.2181
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.9573 - loss: 0.2715


Test Accuracy: 0.97
LSTM model saved to /mnt/data/lstm_email_classifier.h5


In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import AdamW
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

file_path = "/content/Cleaned Smart Email Dataset.csv"
df = pd.read_csv(file_path)

df['Cleaned_Content'] = df['Email Content'].fillna("")

label_encoder = LabelEncoder()
df['Category_Label'] = label_encoder.fit_transform(df['Category'])

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


class EmailDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.long)
        }

dataset = EmailDataset(df['Cleaned_Content'].tolist(), df['Category_Label'].tolist(), tokenizer)


train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

epochs = 3
for epoch in range(epochs):
    model.train()
    loop = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}', leave=True)
    for batch in loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()
        optimizer.step()

        loop.set_postfix(loss=loss.item())


model_path = "/mnt/data/bert_email_classifier.pth"
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)

        correct += (predictions == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f"Validation Accuracy: {accuracy:.2f}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch 